# Notification Webhook

Reads job run outcome via Databricks SDK and sends styled HTML email (success/failure).

In [ ]:
# Widgets: optional values from upstream task
dbutils.widgets.text("received_message", "", "From run_pipeline")
dbutils.widgets.text("job_id", "", "Job ID")
dbutils.widgets.text("run_id", "", "Run ID")

dbutils.widgets.text("job_status", "")
job_status = dbutils.widgets.get("job_status")

In [ ]:
import os
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.jobs import RunResultState
from pyspark.sql import SparkSession

HTML_EMAIL_TEMPLATE = """
<html>
  <head>
    <style>
      body {{ font-family: Arial, sans-serif; margin: 0; padding: 0; background-color: #f4f4f4; color: #333333; }}
      .email-container {{ width: 95%; margin: auto; background-color: #ffffff; border-radius: 6px; padding: 10px 20px; box-shadow: 0 2px 8px rgba(0,0,0,0.05); }}
      .alert-box {{ background-color: #0099D8; color: white; padding: 16px; text-align: center; font-size: 16px; font-weight: bold; border-radius: 4px; margin-bottom: 20px; }}
      .alert-box.failed {{ background-color: #c0392b; }}
      .content pre {{ white-space: pre-wrap; font-family: inherit; font-size: 13px; }}
      .footer {{ padding: 16px 0; font-size: 12px; color: #777777; text-align: center; }}
    </style>
  </head>
  <body>
    <div class="email-container">
      <div class="alert-box {alert_class}">{alert_message}</div>
      <div class="content">{content}</div>
      <div class="footer">Sent via Databricks pipeline • {timestamp}</div>
    </div>
  </body>
</html>
"""

def _build_html_body(alert_message, content_html, alert_class=""):
    now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    return HTML_EMAIL_TEMPLATE.format(alert_message=alert_message, content=content_html, alert_class=alert_class, timestamp=now)

def _get_spark_conf(key, default=""):
    try:
        return spark.conf.get(key, default)
    except Exception:
        return default

spark = SparkSession.builder.getOrCreate()
# Secret scope containing SMTP_HOST, SMTP_PORT, SMTP_PASSWORD (override via env if different)
SMTP_HOST = dbutils.secrets.get(scope="andrea_tardif_smtp_scope", key="SMTP_HOST")
SMTP_PORT = int(dbutils.secrets.get(scope="andrea_tardif_smtp_scope", key="SMTP_PORT"))
SMTP_PASSWORD = dbutils.secrets.get(scope="andrea_tardif_smtp_scope", key="SMTP_PASSWORD")

SMTP_USER = os.environ.get("SMTP_USER", "andrea.tardif16@gmail.com")
ALERT_TO = os.environ.get("ALERT_RECIPIENT", "andrea.tardif@databricks.com")
CATALOG = os.environ.get("CATALOG", "andrea_tardif")

WORKSPACE_URL = _get_spark_conf("spark.databricks.workspaceUrl", "")

JOB_ID = dbutils.widgets.get("job_id") or _get_spark_conf("spark.databricks.job.id", "")
RUN_ID = dbutils.widgets.get("run_id") or _get_spark_conf("spark.databricks.job.runId", "")

In [ ]:
def _parse_job_ids():
    try:
        if not RUN_ID or not str(RUN_ID).strip() or not JOB_ID or not str(JOB_ID).strip():
            return None, None
        r, j = int(str(RUN_ID).strip()), int(str(JOB_ID).strip())
        if r < 1 or j < 1:
            return None, None
        return r, j
    except (ValueError, TypeError):
        return None, None

_run_id, _job_id = _parse_job_ids()
run, job = None, None
if _run_id is None or _job_id is None:
    print("[NOTIFY] Not running in a Databricks job (runId/id not available or invalid) — skipping notification.")
else:
    try:
        client = WorkspaceClient()
        run = client.jobs.get_run(run_id=_run_id)
        job = client.jobs.get(job_id=_job_id)
    except Exception as e:
        print(f"[NOTIFY] Could not fetch run/job from API: {e} — skipping notification.")

In [ ]:
# Send failure email based on job_status (this notebook runs only on failure path)
job_name = job.settings.name if job is not None else f"Job {JOB_ID}"
run_url = f"{WORKSPACE_URL}#job/{JOB_ID}/run/{RUN_ID}"
state_message = (run.state.state_message or "") if run is not None else "Pipeline task failed or did not set status."
task_summaries = []
if run is not None and run.tasks:
    for task in run.tasks:
        task_summaries.append(f"  • {task.task_key}: {task.state.result_state} " +
            (f"(error: {task.state.state_message})" if task.state.state_message else "(ok)"))
task_summary_text = "\n".join(task_summaries) if task_summaries else "  (no task details)"

now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
subject = f"❌ [{job_name}] Run {RUN_ID} — FAILED"
alert_message = "Pipeline run failed."
alert_class = "failed"
content_html = f"<p><strong>Job:</strong> {job_name}<br><strong>Run ID:</strong> {RUN_ID}<br><a href=\"{run_url}\">Open run</a></p><p><strong>Error</strong></p><pre>{state_message or 'N/A'}</pre><p><strong>Tasks</strong></p><pre>{task_summary_text}</pre>"
body = _build_html_body(alert_message, content_html, alert_class)
if ALERT_TO:
    msg = MIMEMultipart("alternative")
    msg["From"] = SMTP_USER
    msg["To"] = ALERT_TO
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "html"))
    try:
        with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
            server.ehlo()
            server.starttls()
            server.login(SMTP_USER, SMTP_PASSWORD)
            server.send_message(msg)
            server.quit()
        print(f"✅ Email sent to {ALERT_TO}")
    except Exception as e:
        print(f"❌ Failed to send email: {e}")
else:
    print("[NOTIFY] ALERT_RECIPIENT not set — skipping email.")